<a href="https://colab.research.google.com/github/AbhayTyagi38/busbookingapp/blob/main/Multi_language_Translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install  transformers sentencepiece  gradio

In [ ]:
!pip install langdetect --quiet

In [ ]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
from langdetect import detect
import gradio as gr

In [ ]:
model_name = "facebook/m2m100_418M"
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(model_name)

In [ ]:
language_name = {
    'en': 'English',
    'fr': 'French',
    'de': 'German',
    'es': 'Spanish',
    'hi': 'Hindi',
    'ru': 'Russian',
    'ja': 'Japanese',
    'zh': 'Chinese'
}

In [ ]:
def translate_text(text, tgt_lang):
    try:
        src_lang = detect(text)
        if src_lang not in language_name or tgt_lang not in language_name:
            return "Unsupported language."

        tokenizer.src_lang = src_lang
        encoded = tokenizer(text, return_tensors="pt")
        generated = model.generate(
            **encoded,
            forced_bos_token_id=tokenizer.get_lang_id(tgt_lang)
        )
        output = tokenizer.batch_decode(generated, skip_special_tokens=True)[0]

        return (
            f"Detected Language: {language_name.get(src_lang, src_lang)}\n"
            f"Translated ({language_name.get(tgt_lang)}): {output}"
        )
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
lang_choices = list(language_name.keys())

gr.Interface(
    fn=translate_text,
    inputs=[
        gr.Textbox(label="Enter text to translate"),
        gr.Dropdown(choices=lang_choices, label="Target Language Code")
    ],
    outputs="text",
    title="🌍 AI Powered Multi language Translator",
    description="Auto-detects the input language and translates to the selected language using Meta's M2M100 model."
).launch()